# States

In this tutorial, we are going to see how to handle perceval state representation.

First, the necessary import:

In [1]:
import perceval as pcvl

### BasicStates <a name="basicstates"></a>

In Linear Optical Circuits, photons can have many discrete degrees of freedom, called modes.
It can be the frequency, the polarisation, the position, or all of them.

We represent these degrees of freedom with Fock states. If we have $n$ photons over $m$ modes, the Fock state $|s_1,s_2,...,s_m\rangle$ means we have $s_i$ photons in the $i^{th}$ mode. Note that $\sum_{i=1}^m s_i =n$.

In Perceval, we will use the class `pcvl.BasicState`.

In [2]:
## Syntax of different BasicState (list, string, etc)
bs1 = pcvl.BasicState([0, 2, 0, 1])
bs2 = pcvl.BasicState('|0,2,0,1>')  # Must start with | and end with >

print(bs1.m)  # Number of modes
print(bs1.n)  # Number of photons

if bs1==bs2:
    print("Those are the same states")

## You can iterate on modes
for i, photon_count in enumerate(bs1):
    print(f"There is {photon_count} photon in mode {i} (or equivalently {bs1[i]}).")

4
3
Those are the same states
There is 0 photon in mode 0 (or equivalently 0).
There is 2 photon in mode 1 (or equivalently 2).
There is 0 photon in mode 2 (or equivalently 0).
There is 1 photon in mode 3 (or equivalently 1).


A `BasicState` is actually more than just a Fock state representation.

In [3]:
# There are three kind of BasicStates

# FockState, all photons are indistinguishable
bs1 = pcvl.BasicState([0, 2, 0, 1])  # Or equivalently BasicState("|0,2,0,1>")
print(type(bs1), isinstance(bs1, pcvl.BasicState))

# NoisyFockState, photons with the same tag are indistinguishable, photons with different tags are distinguishable (they will not interact)
bs2 = pcvl.BasicState([0, 2, 0, 1], [0, 1, 0])  # Or equivalently BasicState("|0,{0}{1},0,{0}>"
print(type(bs2), isinstance(bs2, pcvl.BasicState))

# AnnotatedFockState, with custom annotations (not simulable in the general case, needs a conversion to something simulable first)
bs3 = pcvl.BasicState("|0,{lambda:925}{lambda:925.01},0,{lambda:925.02}>")
print(type(bs3), isinstance(bs3, pcvl.BasicState))


<class 'exqalibur.FockState'> True


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. exqalibur.FockState()
    2. exqalibur.FockState(fs: exqalibur.FockState)
    3. exqalibur.FockState(arg0: int)
    4. exqalibur.FockState(s: str)
    5. exqalibur.FockState(s: str, mode_annotation: dict[int, list[str]])
    6. exqalibur.FockState(fs_vec: list[int])
    7. exqalibur.FockState(fs_vec: list[int], mode_annotation: dict[int, list[str]])

Invoked with: [0, 2, 0, 1], [0, 1, 0]

In [4]:
# Basic methods are common between these types
print(bs1 * pcvl.BasicState([1, 2]))  # Tensor product
print(bs1[1:3])  # Slice

|0,2,0,1,1,2>
|2,0>


### StateVector

A `StateVector` is a complex superposition of `BasicState` with the same number of modes.

In [12]:
# StateVectors can be defined using arithmetic on BasicStates and other StateVectors
sv = pcvl.BasicState([0, 2, 0, 1]) + (0.5 + 0.3j) * pcvl.BasicState([1, 0, 1, 1])

# State vectors normalize themselves upon use
print("State vector is normalized upon use: ", sv)

for (basic_state, amplitude) in sv:
    print(basic_state, "has the complex amplitude", amplitude)

# We can also access amplitudes as in a dictionary
print(sv[pcvl.BasicState([0, 2, 0, 1])])

TypeError: unsupported operand type(s) for +: 'exqalibur.FockState' and 'exqalibur.StateVector'

### Distributions

Perceval exposes state distribution classes that act as mixed states. All states defined in these distributions must have the same number of modes.

In [8]:
# The BSDistribution is a collection of FockStates
bsd = pcvl.BSDistribution()
bsd[pcvl.BasicState([1, 2])] = 0.4
bsd[pcvl.BasicState([2, 1])] = 0.2

print(bsd)
print("Number of modes:", bsd.m)

bsd.normalize()  # Not automatic on distributions

# Distributions act much like python dictionaries
for (state, probability) in bsd.items():
    print(state, "has the probability", probability, f"(or equivalently {bsd[state]})")

print("Tensor product")
print(bsd * pcvl.BasicState([1])) # Tensor product, also works between distributions

{
	|1,2>: 0.4
	|2,1>: 0.2
}
2
|1,2> 0.6666666666666666 (or equivalently 0.6666666666666666)
|2,1> 0.3333333333333333 (or equivalently 0.3333333333333333)
{
	|1,2,1>: 0.6666666666666666
	|2,1,1>: 0.3333333333333333
}


In [11]:
# Likewise, SVDistribution is a collection of StateVectors
svd = pcvl.SVDistribution({pcvl.StateVector([1, 2]) : 0.4,
                           pcvl.BasicState([3, 0]) + pcvl.BasicState([2, 1]) : 0.6})

print("Five random samples according to the distribution:", svd.sample(5))

print("Tensor product")
print(svd * pcvl.SVDistribution(bsd))  # Tensor product between distributions

[0.707*|3,0>+0.707*|2,1>, 0.707*|3,0>+0.707*|2,1>, |1,2>, 0.707*|3,0>+0.707*|2,1>, |1,2>]
{
	|1,2,1,2>: 0.26666666666666666
	0.707*|3,0,2,1>+0.707*|2,1,2,1>: 0.19999999999999998
	0.707*|3,0,1,2>+0.707*|2,1,1,2>: 0.39999999999999997
	|1,2,2,1>: 0.13333333333333333
}


Now that you know how to define states in perceval, you are ready to go to the next tutorial : [Circuits](https://perceval.quandela.net/docs/notebooks/Circuit_Tutorial.html).